In [2]:
import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, History, Callback
from sklearn.metrics import classification_report
import numpy as np
import os
import time
from matplotlib import pyplot

Using TensorFlow backend.


In [3]:
def DataSplit(x_total_addr, y_total_addr, val_ratio = 0.2):
    x_total = np.load(x_total_addr)
    y_total = np.load(y_total_addr)
    idx_shuffle = np.array(range(x_total.shape[0]))
    np.random.shuffle(idx_shuffle)
    idx_train = idx_shuffle[:int(x_total.shape[0]*0.8)]
    idx_val = idx_shuffle[int(x_total.shape[0]*0.8):]
    x_train = x_total[idx_train]
    y_train = y_total[idx_train]
    x_val = x_total[idx_val]
    y_val = y_total[idx_val]
    print("x_total:", x_total.shape, "\ny_total:", y_total.shape, 
          "\nx_train length:", x_train.shape, "\ny_train length:", y_train.shape, 
          "\nx_val length:", x_val.shape, "\ny_val length:", y_val.shape)
    return x_train, y_train, x_val, y_val

In [21]:
x_addr = os.path.join("npy", "26feature", "x_total.npy")
y_addr = os.path.join("npy", "26feature", "y_total.npy")
x_train, y_train, x_val, y_val = DataSplit(x_addr, y_addr)

x_total: (1775, 26) 
y_total: (1775,) 
x_train length: (1420, 26) 
y_train length: (1420,) 
x_val length: (355, 26) 
y_val length: (355,)


In [22]:
########### Naive MLP ############
n_feature = x_train.shape[1] #26
n_classes = len(np.unique(y_train)) #5

Initializer=keras.initializers.glorot_normal(seed=None)
######################################## Model architecture ##############################################
model = Sequential()
model.add(Dense(8, input_dim=n_feature, kernel_initializer = Initializer))
model.add(Dropout(0.2))
model.add(Dense(8, kernel_initializer = Initializer))
model.add(Dropout(0.2))
model.add(Dense(n_classes, activation='softmax', kernel_initializer = Initializer))
######################################## Helpers in callbacks ##############################################
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
######################################## Helpers in callbacks ##############################################
tb = TensorBoard(log_dir=os.path.join('tensorboard', 'logs',))
early_stopper = EarlyStopping(patience=20)
csv_logger = CSVLogger(os.path.join('logs', str(time.time()) + '.log'))
checkpointer = ModelCheckpoint(filepath=os.path.join('checkpoints','{epoch:03d}-{val_sparse_categorical_accuracy:.3f}.hdf5'),
                                verbose=1,save_best_only=True)
history=History()
#########################################################################################################
hist = model.fit(x_train, y_train,
          batch_size=8, nb_epoch=500,
          validation_data=(x_val, y_val),
          callbacks=[tb, early_stopper, csv_logger, checkpointer, history])

C:\Users\Yue Ma\Anaconda\lib\site-packages\ipykernel\__main__.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1420 samples, validate on 355 samples
Epoch 1/500
1420/1420 [==============================] - 1s 789us/step - loss: 9.7683 - sparse_categorical_accuracy: 0.3197 - val_loss: 4.8589 - val_sparse_categorical_accuracy: 0.6986

Epoch 00001: val_loss improved from inf to 4.85886, saving model to checkpoints\001-0.699.hdf5
Epoch 2/500
1420/1420 [==============================] - 0s 326us/step - loss: 6.3907 - sparse_categorical_accuracy: 0.5585 - val_loss: 4.8582 - val_sparse_categorical_accuracy: 0.6986

Epoch 00002: val_loss improved from 4.85886 to 4.85819, saving model to checkpoints\002-0.699.hdf5
Epoch 3/500
1420/1420 [==============================] - 0s 336us/step - loss: 5.3610 - sparse_categorical_accuracy: 0.6373 - val_loss: 4.8582 - val_sparse_categorical_accuracy: 0.6986

Epoch 00003: val_loss improved from 4.85819 to 4.85819, saving model to checkpoints\003-0.699.hdf5
Epoch 4/500
1420/1420 [==============================] - 1s 365us/step - loss: 4.9528 - sparse_categor

1420/1420 [==============================] - 0s 320us/step - loss: 2.7223 - sparse_categorical_accuracy: 0.8289 - val_loss: 2.3156 - val_sparse_categorical_accuracy: 0.8563

Epoch 00034: val_loss did not improve from 2.31556
Epoch 35/500
1420/1420 [==============================] - 1s 395us/step - loss: 2.7532 - sparse_categorical_accuracy: 0.8254 - val_loss: 2.3156 - val_sparse_categorical_accuracy: 0.8563

Epoch 00035: val_loss did not improve from 2.31556
Epoch 36/500
1420/1420 [==============================] - 0s 324us/step - loss: 2.7091 - sparse_categorical_accuracy: 0.8289 - val_loss: 2.3156 - val_sparse_categorical_accuracy: 0.8563

Epoch 00036: val_loss did not improve from 2.31556
Epoch 37/500
1420/1420 [==============================] - 0s 336us/step - loss: 2.7142 - sparse_categorical_accuracy: 0.8303 - val_loss: 2.3156 - val_sparse_categorical_accuracy: 0.8563

Epoch 00037: val_loss did not improve from 2.31556
Epoch 38/500
1420/1420 [==============================] - 0s 

Save models afterwards

In [23]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("best.hdf5")
loaded_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
score = loaded_model.evaluate(x_val, y_val, verbose=0, batch_size = 8)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

sparse_categorical_accuracy: 85.63%


In [25]:
y_pred = loaded_model.predict(x_val)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.00      0.00      0.00        14
           2       0.77      1.00      0.87       168
           3       1.00      1.00      1.00        80
           4       1.00      1.00      1.00        56

   micro avg       0.86      0.86      0.86       355
   macro avg       0.55      0.60      0.57       355
weighted avg       0.75      0.86      0.79       355



C:\Users\Yue Ma\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
pyplot.plot(history.history['sparse_categorical_accuracy'])
pyplot.plot(history.history['val_sparse_categorical_accuracy'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper left')
pyplot.show()

In [ ]:
y_pred = loaded_model.predict(x_train)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_train, y_pred))